In [1]:
!git clone https://github.com/VihariKanukollu/CMBA.git

Cloning into 'CMBA'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 56 (delta 11), reused 55 (delta 10), pack-reused 0 (from 0)
Receiving objects: 100% (56/56), 360.63 KiB | 2.44 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [ ]:
!cd /workspace/CMBA && chmod +x setup_runpod.sh
!cd /workspace/CMBA && bash setup_runpod.sh

🚀 Setting up HRM environment on RunPod...
📦 Updating system packages...
debconf: delaying package configuration, since apt-utils is not installed
(Reading database ... 24286 files and directories currently installed.)
Preparing to unpack .../00-python3.10_3.10.12-1~22.04.11_amd64.deb ...
Unpacking python3.10 (3.10.12-1~22.04.11) over (3.10.12-1~22.04.9) ...
Preparing to unpack .../01-libpython3.10-stdlib_3.10.12-1~22.04.11_amd64.deb ...
Unpacking libpython3.10-stdlib:amd64 (3.10.12-1~22.04.11) over (3.10.12-1~22.04.9) ...
Preparing to unpack .../02-python3.10-minimal_3.10.12-1~22.04.11_amd64.deb ...
Unpacking python3.10-minimal (3.10.12-1~22.04.11) over (3.10.12-1~22.04.9) ...
Preparing to unpack .../03-libpython3.10-minimal_3.10.12-1~22.04.11_amd64.deb ...
Unpacking libpython3.10-minimal:amd64 (3.10.12-1~22.04.11) over (3.10.12-1~22.04.9) ...
Preparing to unpack .../04-python3-pkg-resources_59.6.0-1.2ubuntu0.22.04.3_all.deb ...
Unpacking python3-pkg-resources (59.6.0-1.2ubuntu0.22.04.

In [ ]:
%cd HRM
# Quick test dataset (small for debugging)
!python dataset/build_sudoku_dataset.py --output-dir data/sudoku-quick-test --subsample-size 100 --min-difficulty 8 --num-aug 10

# Full training dataset
!python dataset/build_sudoku_dataset.py --output-dir data/sudoku-extreme-1k-aug-1000 --subsample-size 1000 --min-difficulty 8 --num-aug 1000

# Full training datasetv - Maze
!python dataset/build_maze_dataset.py --output-dir data/maze-30x30-hard-1k --aug

# Download ARC datasets
!mkdir -p dataset/raw-data
!git clone https://github.com/fchollet/ARC-AGI.git dataset/raw-data/ARC-AGI

!python dataset/build_arc_dataset.py --output-dir data/arc-aug-1000 --dataset-dirs dataset/raw-data/ARC-AGI/data

In [4]:
!wandb login 1dc0a6d6c0fcfbea7784e7923daf7aba591ac301

wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [5]:
!python3 -c "import torch; print(f'CUDA available: {torch.cuda.is_available()}'); print(f'GPU: {torch.cuda.get_device_name(0)}')"

CUDA available: True
GPU: NVIDIA A40


In [6]:
!python3 -c "import torch, einops, tqdm, wandb, omegaconf; print('✅ All HRM dependencies working!')"

✅ All HRM dependencies working!


In [9]:
%cd /workspace/CMBA/HRM
!pkill -f "python pretrain.py"
!python pretrain.py data_path=data/sudoku-quick-test epochs=100 eval_interval=null global_batch_size=32 lr=7e-5 puzzle_emb_lr=7e-5 weight_decay=1.0 puzzle_emb_weight_decay=1.0

/workspace/CMBA/HRM
Error executing job with overrides: ['data_path=data/sudoku-quick-test', 'epochs=100', 'eval_interval=null', 'global_batch_size=32', 'lr=7e-5', 'puzzle_emb_lr=7e-5', 'weight_decay=1.0', 'puzzle_emb_weight_decay=1.0']
Traceback (most recent call last):
  File "/workspace/CMBA/HRM/models/layers.py", line 8, in <module>
    from flash_attn_interface import flash_attn_func  # type: ignore[import]
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ModuleNotFoundError: No module named 'flash_attn_interface'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/workspace/CMBA/HRM/pretrain.py", line 411, in launch
    train_state = init_train_state(config, train_metadata, world_size=WORLD_SIZE)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspace/CMBA/HRM/pretrain.py", line 177, in init_train_state
    model, optimizers, optimizer_lrs = create_model(config, train_me

In [10]:
!pip uninstall flash-attn -y

Found existing installation: flash_attn 2.8.3
Uninstalling flash_attn-2.8.3:
  Successfully uninstalled flash_attn-2.8.3


In [11]:
!python -c "import torch; print(f'PyTorch: {torch.__version__}'); print(f'CUDA: {torch.version.cuda}')"

PyTorch: 2.8.0.dev20250319+cu128
CUDA: 12.8


In [12]:
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda'
os.environ['PATH'] = f"/usr/local/cuda/bin:{os.environ.get('PATH', '')}"
os.environ['LD_LIBRARY_PATH'] = f"/usr/local/cuda/lib64:{os.environ.get('LD_LIBRARY_PATH', '')}"
os.environ['TORCH_CUDA_ARCH_LIST'] = "8.0;8.6;8.9;9.0"

!pip install ninja packaging wheel

In [13]:
# Cell 3: Install FlashAttention from source (this will take several minutes)
!pip install flash-attn --no-build-isolation --no-cache-dir --force-reinstall --verbose

Using pip 25.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 338.9 MB/s  0:00:00
  Running command python setup.py egg_info
  /usr/local/lib/python3.11/dist-packages/setuptools/__init__.py:94: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
  !!

          ********************************************************************************
          Requirements should be satisfied by a PEP 517 installer.
          If you are using pip, you can try `pip install --use-pep517`.
          ********************************************************************************

  !!
    dist.fetch_build_eggs(dist.setup_requires)
  /usr/local/lib/python3.11/dist-packages/setuptools/dist.py:751: SetuptoolsDeprecationWarning: License classifiers are deprecated.
  !!

          ********************************************************************************
          Please consider removing the follo

In [14]:
# Cell 6: Test the installation
import torch
try:
    from flash_attn import flash_attn_func
    print("✅ FlashAttention installed successfully!")
    print(f"PyTorch: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
except ImportError as e:
    print(f"❌ FlashAttention import failed: {e}")

✅ FlashAttention installed successfully!
PyTorch: 2.8.0+cu128
CUDA available: True


In [ ]:
%cd /workspace/CMBA/HRM
!pkill -f "python pretrain.py"
!python pretrain.py data_path=data/sudoku-quick-test epochs=100 eval_interval=null global_batch_size=32 lr=7e-5 puzzle_emb_lr=7e-5 weight_decay=1.0 puzzle_emb_weight_decay=1.0

In [ ]:
!DISABLE_COMPILE=1 python pretrain.py \
  arch=hrm_4l_v3 data_path=data/sudoku-quick-test epochs=100 eval_interval=null global_batch_size=32 lr=7e-5 puzzle_emb_lr=7e-5 weight_decay=1.0 puzzle_emb_weight_decay=1.0

In [ ]:
DISABLE_COMPILE=1 python pretrain.py \
  arch=hrm_4l_v3 data_path=data/sudoku-quick-test epochs=100 eval_interval=null \
  global_batch_size=32 lr=7e-5 puzzle_emb_lr=7e-5 weight_decay=1.0 puzzle_emb_weight_decay=1.0 \
  arch.cross_attn_in_settle=true arch.graph_in_settle=true arch.graph_top_k=0

In [ ]:
#Default config V3

GRAD_CLIP_NORM=1.0 \
python HRM/pretrain.py \
  arch=hrm_4l_v3 data_path=data/sudoku-quick-test epochs=300 eval_interval=null \
  global_batch_size=32 lr=1e-4 lr_min_ratio=0.1 lr_warmup_steps=1000 \
  puzzle_emb_lr=1e-4 weight_decay=0.01 puzzle_emb_weight_decay=0.1 \
  arch.halt_max_steps=8 arch.A_cycles=2 arch.M_per_B=2 arch.B_per_A=1 \
  arch.C_layers=2 arch.M_layers=2 arch.B_layers=2 arch.A_layers=2 \
  +arch.rms_norm_eps=1e-5 arch.use_film=false arch.rope_factor=1.0

In [ ]:
#Default config V3 with console log

DISABLE_COMPILE=1 GRAD_CLIP_NORM=1.0 \
python HRM/pretrain.py \
arch=hrm_4l_v3 data_path=data/sudoku-quick-test epochs=300 eval_interval=null \
global_batch_size=32 lr=1e-4 lr_min_ratio=0.1 lr_warmup_steps=1000 \
puzzle_emb_lr=1e-4 weight_decay=0.01 puzzle_emb_weight_decay=0.1 \
arch.halt_max_steps=8 arch.A_cycles=2 arch.M_per_B=2 arch.B_per_A=1 \
arch.C_layers=2 arch.M_layers=2 arch.B_layers=2 arch.A_layers=2 \
+arch.rms_norm_eps=1e-5 arch.use_film=false arch.rope_factor=1.0 \
checkpoint_every_eval=false +checkpoint_path=null \
+console_log_eval=true +console_log_max_items=50